In [1]:
from sahi.utils.detectron2 import Detectron2TestConstants

# import required functions, classes
from sahi import AutoDetectionModel
from sahi.predict import get_sliced_prediction
from IPython.display import Image

In [2]:
# setup all the models configurations for that evaluation experiment
models_configs = [
    {
        "model_name": "mask_rcnn_R_50_FPN_3x",
        "checkpoint": "/home/etaylor/code_projects/thesis/checkpoints/detectron2/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/29-04-2024_16-02-00/model_final.pth",
        "yaml_file": "/home/etaylor/code_projects/thesis/checkpoints/detectron2/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/29-04-2024_16-02-00/config.yaml"
    },
    {
        "model_name": "mask_rcnn_X_101_32x8d_FPN_3x",
        "checkpoint": "/home/etaylor/code_projects/thesis/checkpoints/detectron2/COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x/27-04-2024_13-54-23/model_final.pth",
        "yaml_file": "/home/etaylor/code_projects/thesis/checkpoints/detectron2/COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x/27-04-2024_13-54-23/config.yaml"
    },
    {
        "model_name": "faster_rcnn_R_50_C4_1x",
        "checkpoint": "/home/etaylor/code_projects/thesis/checkpoints/detectron2/COCO-Detection/faster_rcnn_R_50_C4_1x/29-04-2024_16-09-41/model_final.pth",
        "yaml_file": "/home/etaylor/code_projects/thesis/checkpoints/detectron2/COCO-Detection/faster_rcnn_R_50_C4_1x/29-04-2024_16-09-41/config.yaml"
    },
    {
        "model_name": "faster_rcnn_R_50_DC5_1x",
        "checkpoint": "/home/etaylor/code_projects/thesis/checkpoints/detectron2/COCO-Detection/faster_rcnn_R_50_DC5_1x/29-04-2024_16-09-55/model_final.pth",
        "yaml_file": "/home/etaylor/code_projects/thesis/checkpoints/detectron2/COCO-Detection/faster_rcnn_R_50_DC5_1x/29-04-2024_16-09-55/config.yaml"
    },
]

In [3]:
model_path = models_configs[0]['checkpoint']
config_path = models_configs[0]['yaml_file']
detection_model = AutoDetectionModel.from_pretrained(
    model_type='detectron2',
    model_path=model_path,
    config_path=config_path,
    confidence_threshold=0.5,
    image_size=512,
    device="cuda:0", # or 'cpu'
)

05/03/2024 19:18:05 - WARNING - fvcore.common.config -   Loading config /home/etaylor/code_projects/thesis/checkpoints/detectron2/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/29-04-2024_16-02-00/config.yaml with yaml.unsafe_load. Your machine may be at risk if the file contains malicious content.


/home/etaylor/code_projects/thesis/checkpoints/detectron2/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/29-04-2024_16-02-00/config.yaml not available in Model Zoo!


05/03/2024 19:18:11 - INFO - detectron2.checkpoint.detection_checkpoint -   [DetectionCheckpointer] Loading from /home/etaylor/code_projects/thesis/checkpoints/detectron2/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/29-04-2024_16-02-00/model_final.pth ...
05/03/2024 19:18:11 - INFO - fvcore.common.checkpoint -   [Checkpointer] Loading from /home/etaylor/code_projects/thesis/checkpoints/detectron2/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/29-04-2024_16-02-00/model_final.pth ...
05/03/2024 19:18:11 - WARNING - sahi.models.detectron2 -   Attribute 'thing_classes' does not exist in the metadata of dataset 'etaylor/cannabis_patches_train_26-04-2024_15-44-44': metadata is empty.


In [37]:
import pandas as pd
# load the test df csv
test_df_path = "/home/etaylor/code_projects/thesis/data/test_data_01-05-2024_11-57-23.csv"
test_df = pd.read_csv(test_df_path)
print(f"Number of records in the test df: {len(test_df)}")
test_df.head(10)

Number of records in the test df: 10


,image_number,image_path,week_number,clear,cloudy,amber
0,IMG_0542,/sise/home/etaylor/images/raw_images/week6_22_...,week6_22_05_2023,36,49,0
1,IMG_2198,/sise/home/etaylor/images/raw_images/week9_15_...,week9_15_06_2023,48,148,10
2,IMG_1753,/sise/home/etaylor/images/raw_images/week8_07_...,week8_07_06_2023,74,21,0
3,IMG_1093,/sise/home/etaylor/images/raw_images/week7_01_...,week7_01_06_2023,33,143,36
4,IMG_0019,/sise/home/etaylor/images/raw_images/week5_18_...,week5_18_05_2023,144,100,4
5,IMG_1096,/sise/home/etaylor/images/raw_images/week7_01_...,week7_01_06_2023,30,121,39
6,IMG_2271,/sise/home/etaylor/images/raw_images/week9_15_...,week9_15_06_2023,23,58,12
7,IMG_2285,/sise/home/etaylor/images/raw_images/week9_15_...,week9_15_06_2023,30,128,12
8,IMG_1079,/sise/home/etaylor/images/raw_images/week7_01_...,week7_01_06_2023,68,79,6
9,IMG_0058,/sise/home/etaylor/images/raw_images/week5_18_...,week5_18_05_2023,78,31,0


In [46]:
import time
sahi_results = {}
for i in range(len(test_df)):
    image_path = test_df.iloc[i]['image_path']
    image_number = test_df.iloc[i]['image_number']
    print(f"Processing image: {image_number}")
    start = time.time()
    result = get_sliced_prediction(
    image_path,
    detection_model,
    slice_height = 512,
    slice_width = 512,
    overlap_height_ratio = 0,
    overlap_width_ratio = 0,
    verbose=True,
    )
    end_time = time.time() - start
    print(f"Time taken to process image: {end_time}")

    start = time.time()
    result.export_visuals(export_dir="/home/etaylor/code_projects/thesis/src/segmentation/notebooks/detectron2/sahi_results/",
                          text_size=1,
                          rect_th=1,
                          hide_labels=True,
                          hide_conf=True,
                          file_name=image_number
    )
    end = time.time() - start
    print(f"Time taken to export visuals: {end}")
    sahi_results[image_number]= result

Processing image: IMG_0542


Performing prediction on 48 number of slices.
Time taken to process image: 41.96157431602478
Time taken to export visuals: 83.98481011390686
Processing image: IMG_2198
Performing prediction on 48 number of slices.
Time taken to process image: 68.78479552268982
Time taken to export visuals: 147.46600651741028
Processing image: IMG_1753
Performing prediction on 48 number of slices.
Time taken to process image: 33.84622001647949
Time taken to export visuals: 70.01870894432068
Processing image: IMG_1093
Performing prediction on 48 number of slices.
Time taken to process image: 38.31111526489258
Time taken to export visuals: 79.76271843910217
Processing image: IMG_0019
Performing prediction on 48 number of slices.
Time taken to process image: 73.16555595397949
Time taken to export visuals: 133.9997091293335
Processing image: IMG_1096
Performing prediction on 48 number of slices.
Time taken to process image: 52.48532676696777
Time taken to export visuals: 113.17336201667786
Processing image:

In [47]:
def get_trichome_distribution(object_prediction_list):
    # Initialize the distribution dictionary
    trichome_dist = {
        "clear": 0,
        "cloudy": 0,
        "amber": 0,
    }
    
    # Iterate through each object in the prediction list
    for obj in object_prediction_list:
        # Increment the count based on the category ID
        if obj.category.id == 1:
            trichome_dist["clear"] += 1
        elif obj.category.id == 2:
            trichome_dist["cloudy"] += 1
        elif obj.category.id == 3:
            trichome_dist["amber"] += 1
    
    return trichome_dist

In [48]:
trichome_distribution = {}
for image_number in sahi_results:
    result = sahi_results[image_number]
    trichome_dist = get_trichome_distribution(result.object_prediction_list)
    print(f"Image Number: {image_number}")
    print(f"Trichome Distribution: {trichome_dist}")
    trichome_distribution[image_number] = trichome_dist

Image Number: IMG_0542
Trichome Distribution: {'clear': 46, 'cloudy': 136, 'amber': 3}
Image Number: IMG_2198
Trichome Distribution: {'clear': 27, 'cloudy': 281, 'amber': 10}
Image Number: IMG_1753
Trichome Distribution: {'clear': 111, 'cloudy': 50, 'amber': 0}
Image Number: IMG_1093
Trichome Distribution: {'clear': 47, 'cloudy': 116, 'amber': 9}
Image Number: IMG_0019
Trichome Distribution: {'clear': 125, 'cloudy': 157, 'amber': 1}
Image Number: IMG_1096
Trichome Distribution: {'clear': 37, 'cloudy': 162, 'amber': 40}
Image Number: IMG_2271
Trichome Distribution: {'clear': 29, 'cloudy': 103, 'amber': 10}
Image Number: IMG_2285
Trichome Distribution: {'clear': 39, 'cloudy': 199, 'amber': 11}
Image Number: IMG_1079
Trichome Distribution: {'clear': 113, 'cloudy': 137, 'amber': 5}
Image Number: IMG_0058
Trichome Distribution: {'clear': 71, 'cloudy': 57, 'amber': 3}


In [44]:
test_df.head(10)

,image_number,image_path,week_number,clear,cloudy,amber
0,IMG_0542,/sise/home/etaylor/images/raw_images/week6_22_...,week6_22_05_2023,36,49,0
1,IMG_2198,/sise/home/etaylor/images/raw_images/week9_15_...,week9_15_06_2023,48,148,10
2,IMG_1753,/sise/home/etaylor/images/raw_images/week8_07_...,week8_07_06_2023,74,21,0
3,IMG_1093,/sise/home/etaylor/images/raw_images/week7_01_...,week7_01_06_2023,33,143,36
4,IMG_0019,/sise/home/etaylor/images/raw_images/week5_18_...,week5_18_05_2023,144,100,4
5,IMG_1096,/sise/home/etaylor/images/raw_images/week7_01_...,week7_01_06_2023,30,121,39
6,IMG_2271,/sise/home/etaylor/images/raw_images/week9_15_...,week9_15_06_2023,23,58,12
7,IMG_2285,/sise/home/etaylor/images/raw_images/week9_15_...,week9_15_06_2023,30,128,12
8,IMG_1079,/sise/home/etaylor/images/raw_images/week7_01_...,week7_01_06_2023,68,79,6
9,IMG_0058,/sise/home/etaylor/images/raw_images/week5_18_...,week5_18_05_2023,78,31,0
